In [24]:
# visualize important points optical flow

import os, json, math
from collections import Counter
import cv2
import numpy as np

root = "analysis/optical-flow/"
fns = [root + f for f in os.listdir(root) if f[0] != "."][7:]

for flow_fn in fns:
    with open(flow_fn) as f: 
        j = json.load(f)
        
        fn = "videos/" + flow_fn.split(".json")[0].split("/")[-1] + ".mp4"
        print fn
        
        cap = cv2.VideoCapture(fn)
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Create some random colors
        color = np.random.randint(0,255,(100,3))
        ret, old_frame = cap.read()
        # mask = np.zeros_like(old_frame)

        frame_count = 0
        unique_frames = sorted(list(set([c["frame"] for c in j])))

        while(1):
            frame_count += 1

            ret,frame = cap.read()
            
            lower_frames = [u for u in unique_frames if u <= frame_count]
            upper_frames = [u for u in unique_frames if u > frame_count]
            if not lower_frames: 
                lower_frames = [0]
            if not upper_frames:
                upper_frames = [10000]
                
            lower_bound_frame = lower_frames[-1]
            upper_bound_frame = upper_frames[0]

            if frame != None and frame.any() and frame_count % 5: 
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                rel_feats = [feat for feat in j if feat["frame"] == lower_bound_frame]
                for feat in rel_feats:
                    
                    a = int(feat["b"][0])
                    b = int(feat["b"][1])
                    c = int(feat["a"][0])
                    d = int(feat["a"][1])
                    
                    # mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
                    frame = cv2.circle(frame,(a,b),5,color[int(feat["feature"])].tolist(),-1)
                    
                img = frame
                cv2.imshow('frame',img)
            
                k = cv2.waitKey(30) & 0xff
                if k == 27:
                    break

        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        cap.release()

videos/probation-reg-small.mp4


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:44: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


KeyboardInterrupt: 

In [ ]:
# find clusters from optical flow files

import os, json, math
from collections import Counter,defaultdict
import cv2
import numpy as np

from sklearn.cluster import KMeans

root = "analysis/optical-flow/"
fns = [root + f for f in os.listdir(root) if f[0] != "."][1:]
clusters = defaultdict(dict)

for flow_fn in fns:
    with open(flow_fn) as f: 
        j = json.load(f)
        print flow_fn
        
        unique_frames = sorted(list(set([k["frame"] for k in j])))
        
        clusters[flow_fn] = []
        
        for frame in unique_frames: 
            rel = [k for k in j if k["frame"] == frame]
            points = np.array([(k["b"][0], k["b"][1]) for k in rel])
            
            # print np.array(points[0:20])

            if len(points) > 1:
                km = KMeans(n_clusters=2, random_state=0).fit(points)
                labels = km.labels_
                cl1 = [points[i] for i in range(len(points)) if i in np.where(labels == 0)[0]]
                cl2 = [points[i] for i in range(len(points)) if i in np.where(labels == 1)[0]]
                
                # also want to record which features comprise the clusters
                cl_features = [[],[]]
                cl_list = [cl1, cl2]
                
                for i in range(len(cl_list)):
                    cl = cl_list[i]
                    for p in cl: 
                        rel_feat = [k for k in rel if k["b"][0] == p[0] and k["b"][1] == p[1]][0]["feature"]
                        cl_features[i].append(rel_feat)
                
                
                clusters[flow_fn].append({
                        "cluster-centers": km.cluster_centers_,
                        "scores": [np.std(cl1), np.std(cl2), km.score(points)],
                        "frame": frame,
                        "cluster-features": cl_features,
                    })       

# turns out what we actually want to do is cluster features together for the whole movie
# not to cluster together features in each frame

In [6]:
clusters["analysis/optical-flow/equation-small.json"]

[{'cluster-centers': array([[ 409.35868835,  158.5582962 ],
         [ 477.42402954,  213.89624939]]),
  'frame': 29,
  'scores': [126.51721304553926, 132.40169501347756, -2810.4452298578108]},
 {'cluster-centers': array([[ 354.49961045,  159.67385146],
         [ 171.51803373,  147.74359538]]),
  'frame': 58,
  'scores': [111.51496126066371, 49.216091147492101, -237029.85366509156]},
 {'cluster-centers': array([[ 292.99374945,  129.5480527 ],
         [ 230.16553752,  155.16824849]]),
  'frame': 87,
  'scores': [81.827348900731764, 39.166225314929186, -1143.0729974563146]},
 {'cluster-centers': array([[ 296.11710982,  130.21087022],
         [ 220.58319528,  145.85986764]]),
  'frame': 116,
  'scores': [83.023065374798676, 38.172291659132433, -1112.6235809101345]},
 {'cluster-centers': array([[ 303.47573547,  129.57158356],
         [ 217.06578827,  145.59887505]]),
  'frame': 145,
  'scores': [87.037571920407018, 36.478577352748211, -1158.4151952475368]},
 {'cluster-centers': array([

In [21]:
# visualize cluster centers

root = "analysis/optical-flow/"
fns = [root + f for f in os.listdir(root) if f[0] != "."][7:]

for flow_fn in fns:
    with open(flow_fn) as f: 
        flow_fn_root = flow_fn.split(".json")[0].split("/")[-1]
        
        j = clusters[flow_fn]
        fn = "videos/" + flow_fn_root + ".mp4"
        print fn
        
        cap = cv2.VideoCapture(fn)
        fps = cap.get(cv2.CAP_PROP_FPS)

        # Create some random colors
        color = np.random.randint(0,255,(100,3))
        ret, old_frame = cap.read()
        # mask = np.zeros_like(old_frame)

        frame_count = 0
        unique_frames = sorted(list(set([c["frame"] for c in j])))

        while(1):
            frame_count += 1

            ret,frame = cap.read()
            
            lower_frames = [u for u in unique_frames if u <= frame_count]
            upper_frames = [u for u in unique_frames if u > frame_count]
            if not lower_frames: 
                lower_frames = [0]
            if not upper_frames:
                upper_frames = [10000]
                
            lower_bound_frame = lower_frames[-1]
            upper_bound_frame = upper_frames[0]

            if frame != None and frame.any() and frame_count % 5: 
                frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                rel_feats = [feat for feat in j if feat["frame"] == lower_bound_frame]
                cluster_centers = []
                print len(rel_feats)
                if rel_feats: 
                    cluster_centers = rel_feats[0]["cluster-centers"]
                
                cluster_center_count = 0
                for cluster_center in cluster_centers:
                    
                    a = int(cluster_center[0])
                    b = int(cluster_center[1])
                    
                    # mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
                    frame = cv2.circle(frame,(a,b),5,color[int(cluster_center_count)].tolist(),-1)
                    cluster_center_count +=1
                    
                img = frame
                cv2.imshow('frame',img)
            
                k = cv2.waitKey(30) & 0xff
                if k == 27:
                    break

        cv2.waitKey(1)
        cv2.destroyAllWindows()
        cv2.waitKey(1)
        cap.release()

videos/probation-reg-small.mp4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:40: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


KeyboardInterrupt: 

In [10]:
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)
cap.release()

In [23]:
print fps

29.4117647059
